<a href="https://colab.research.google.com/github/amalkhairin/Genetic-Algorithm/blob/main/GeneticAlgorithm_1301180248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

init population

In [ ]:
import numpy as np

def init_pop(pop_size, chrom_size):
    #generate bilangan random int dari 0 - 10, dengan size sebesar pop_size * chrom_size
    populasi = np.random.randint(0,10, (pop_size, chrom_size))
    return populasi

decode chromosom

In [ ]:
def decode_chrom(populasi,min,max):
    pop_size, chrom_size = populasi.shape
    jumlah_x = len(min)
    alele = chrom_size // jumlah_x
    temp_populasi = populasi.copy().reshape((pop_size, jumlah_x, alele))

    decimal = np.array( [10**-(i+1) for i in range(alele)] )
    scale = (max-min) / (9*np.sum(decimal))

    decoded_populasi = min + (scale * temp_populasi.dot(decimal))

    return decoded_populasi

fitness evaluation

In [ ]:
def fx(x):
    #x berupa array
    x1, x2 = x
    a = np.cos(x1) * np.sin(x2)
    b = x1 // (x2**2 + 1)
    c = a - b

    return c


In [ ]:
def fitness_eva(decoded_populasi, a=0.1):
    #minimasi fungsi dengan 1/h+a
    fitness = 1/(fx(decoded_populasi.T)+a)

    return fitness


Roulette wheel (parent selection)

In [ ]:
def roulette_wheel(fitness):
    parent_id = []
    probability = fitness.copy()
    probability = probability / np.sum(probability)

    while len(parent_id) <2:
        r = np.random.rand()
        t, i = 0, 0
        while t< r:
            t += probability[i]
            i += 1
        parent_id.append(i-1)
    return parent_id

crossover

In [ ]:
def crossover(parent_chromosom, Pc = 0.8):
    _, chrom_size = parent_chromosom.shape

    child_chromosom = parent_chromosom.copy()

    r = np.random.rand()
    if r <= Pc:
        #titik potong
        p = np.random.randint(0, chrom_size)                 
        #pindah silang
        child_chromosom[::-1, p:] = parent_chromosom[::, p:]  
    
    return child_chromosom

mutation

In [ ]:
def mutation(chromosom, Pm = 0.1):
    r= np.random.rand( *chromosom.shape)

    mutation = np.random.randint(0, 10, (chromosom.shape))
    mutated_chromosom = np.where(r >= Pm, chromosom, mutation)

    return mutated_chromosom

elitism (survivor selection)

In [ ]:
def elitisme(populasi, fitness):
    best_id = np.argmax(fitness)
    best_chromosom = populasi[best_id].copy()
    
    #memasukkan kromosom dengan fitness terbaik ke populasi
    new_populasi = np.vstack([best_chromosom, best_chromosom])

    return new_populasi

main

In [ ]:
min = np.array([-1, -1])    # x1 x2 min
max = np.array([2, 1])      # x1 x2 max

pop_size = 300      # jumlah populasi
max_gen = 300       # max gen
chrom_size = 10     #jumlah kromosom
Pc = 0.9
Pm = 0.1

#menampung history fitness terbaik
history = []

#init populasi
populasi = init_pop(pop_size, chrom_size)

for g in range(max_gen):
    #decode chromosom
    decoded_populasi = decode_chrom(populasi, min, max)
    #fitness evaluation
    fitness = fitness_eva(decoded_populasi, a=3)

    #survivor selection
    new_populasi = elitisme(populasi, fitness)
    #menyimpan fitness ke history
    history.append(np.max(fitness))

    #check generasi dan fitness
    if(g % 10 == 0):
        print('generasi: {0}, fitness: {1}'.format(g, np.max(fitness)))

    while new_populasi.shape[0] < pop_size:
        #parent selection dengan Roulette wheel
        parent_id = roulette_wheel(fitness)
        parent_chromosom = populasi[parent_id].copy()

        #crossover
        child_chromosom = crossover(parent_chromosom, Pc)
        #mutation
        mutated_chromosom = mutation(child_chromosom, Pm)

        new_populasi = np.vstack([new_populasi, mutated_chromosom])

    populasi = new_populasi         #assign populasi baru

In [ ]:
decoded_populasi = decode_chrom(populasi, min, max)
fitness = fitness_eva(decoded_populasi, a=3)
best_id = np.argmax(fitness)

#print solusi terbaik
print('''x1= {0}
x2= {1}
fx= {2}
fitness = {3}'''. format(decoded_populasi[best_id][0], decoded_populasi[best_id][1],
                   fx(decoded_populasi[best_id]), np.max(fitness))
)